# UnionML Demo

In [ ]:
from pictionary_app import model

num_classes = 3

# run training on a remote
execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=1000,
    class_limit=num_classes,
)

In [ ]:
# wait for the execution to complete, then a
model.remote_load(execution)

In [ ]:
import gradio as gr

gr.Interface(
    fn=model.predict,
    inputs="sketchpad",
    outputs="label",
    live=True,
    allow_flagging="never",
).launch()

# Flytekit Demo

In [ ]:
# - get the features and labels locally
# - get model.artifact.model_object
# - invoke the Flyte workflow for predictions
# - visualize the original images, with accuracy score

In [ ]:
from pictionary_app.dataset import QuickDrawDataset

dataset = QuickDrawDataset("/tmp/quickdraw_data", max_items_per_class=1000, class_limit=num_classes)
dataset

In [ ]:
model.artifact.model_object

Create a `FlyteRemote` object and fetch the batch prediction workflow

In [ ]:
from flytekit.remote import FlyteRemote
from flytekit.configuration import Config


remote = FlyteRemote(config=Config.auto(config_file="./config/config-remote.yaml"), data_upload_location="s3://open-compute-playground/data")
wf = remote.fetch_workflow(project="unionml", domain="development", name="flytekit_demo.batch_predictions.wf", version="v1")

Get features and labels

In [ ]:
feature_list = []
label_list = []
for i in range(5):
    X, y = dataset[i]
    feature_list.append(X.squeeze())
    label_list.append(y)

In [ ]:
# NOTE: make sure you have aws credentials for the s3://open-compute-playground bucket
execution = remote.execute(
    wf,
    inputs={"model_object": model.artifact.model_object, "feature_list": feature_list},
    project="unionml",
    domain="development",
    wait=True,
)

In [ ]:
execution